# Utility Functions to get test data easily

In [ ]:
101160

In [6]:
import sys
print(sys.path)
sys.path.insert(0, '/workspace/')
import numpy as np
from spacy import prefer_gpu
import torch
from spacy.util import minibatch, compounding
from spacy.training import offsets_to_biluo_tags
import json
import spacy
import pandas as pd
from spacy.training import Example
import random
import warnings
from qrytool import load_data_into_dataframe, insert_dataframe_into_table


if prefer_gpu():
    print("GPU is being used.")
else:
    print("CPU is being used.")
# 새 모델 생성 또는 기존 모델 로드
nlp = spacy.blank('ko')  # 새 모델을 생성하거나
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# nlp = to_device(nlp, device)
# NER 파이프라인 추가
if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')



# 저장된 모델 로드
nlp = spacy.load("/workspace/ko_addr_ner_model_999/")

['/', '/opt/conda/envs/py39/lib/python39.zip', '/opt/conda/envs/py39/lib/python3.9', '/opt/conda/envs/py39/lib/python3.9/lib-dynload', '', '/opt/conda/envs/py39/lib/python3.9/site-packages', '/opt/conda/envs/py39/lib/python3.9/site-packages/shap-0.42.1-py3.9-linux-x86_64.egg', '/opt/conda/envs/py39/lib/python3.9/site-packages/slicer-0.0.7-py3.9.egg']
GPU is being used.


In [7]:
from fuzzywuzzy import process
import ipywidgets as widgets
from IPython.display import display
import warnings
from qrytool import load_data_into_dataframe, insert_dataframe_into_table
from IPython.display import clear_output
warnings.simplefilter("ignore")
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
# def find_closest_building(input_name, building_list):
#     # 건물명 리스트와 입력된 건물명을 비교하여 가장 유사한 결과를 찾습니다.
#     # 'limit' 매개변수는 반환하고자 하는 가장 유사한 결과의 개수입니다.
#     closest_match = process.extractOne(input_name, building_list)
#     return closest_match



def search_complex_price(complex_no,dong_name, ho_name):
    qry="""
        SELECT
            nc.complex_name,
            ncp.complex_no,
            ncd.build_name,
            ncd.ho_name,
            ncd.ho_floor,
            ncp.pyeong_no,
            ncp.deal_upper_price_limit,
            ncp.base_year_month_day,
            ncp.deal_upper_price_limit,
            ncp.deal_average_price,
            ncp.deal_low_price_limit,
            ncp.deal_average_price_change_amount,
            ncp.lease_upper_price_limit,
            ncp.lease_average_price,
            ncp.lease_low_price_limit,
            ncp.lease_average_price_change_amount,
            ncp.upper_price_limit,
            ncp.average_price_limit,
            ncp.low_price_limit,
            ncp.price_change_amount,
            ncp.lease_per_deal_rate,
            ncp.rent_low_price,
            ncp.deposit,
            ncp.rent_upper_price
        FROM naver_complexes  nc
        LEFT JOIN naver_complex_dongho ncd ON nc.complex_no = ncd.complex_no
        LEFT JOIN naver_complex_prices ncp ON ncp.complex_no = nc.complex_no and ncp.pyeong_no = ncd.pyeong_no
        WHERE
    """
    dong_name=dong_name.replace("동","") if dong_name else ''
    ho_name=ho_name.replace("호","") if ho_name else ''
    target_qry=qry+f"nc.complex_no='{complex_no}'"
    if dong_name and dong_name.strip()!='':
        target_qry+=f"and build_name='{dong_name}'"
    else:
        target_qry+="and build_name='1'"

    if ho_name and ho_name.strip()!='':
        target_qry+=f"and ho_name='{ho_name}'"
    # print(target_qry)
    df = load_data_into_dataframe(target_qry)
    display(df) #dong ho pyeong and price
    return df


def find_closest_building(input_name, df):
    # 건물명 리스트와 입력된 건물명을 비교하여 가장 유사한 결과를 찾습니다.
    # 'limit' 매개변수는 반환하고자 하는 가장 유사한 결과의 개수입니다.
    closest_matches = process.extract(input_name, df['complex_name'], limit=3)
    closest_df = pd.DataFrame()

    for match in closest_matches:
        match_df = df[df['complex_name'] == match[0]]
        match_df['similarity'] = match[1]
        closest_df = pd.concat([closest_df, match_df])

    display(closest_df) # top 3 similar apt
    # Filter the dataframe to only include rows with similarity greater than 85
    closest_df = closest_df[closest_df['similarity'] > 85]
    closest_df = closest_df.loc[closest_df['similarity'].idxmax()]
    return closest_df[['complex_no', 'complex_name', 'latitude','longitude','real_estate_type_name', 'similarity']]


def find_complex_price(address):
    closest_df=None
    if address is not None:
    # Analyze the address with the NLP model
        doc = nlp(address.strip())
        # Create a dictionary to store the entities for this address
        entities = {'address': address.strip()}
        # Add the entities to the dictionary
        for ent in doc.ents:
            entities[ent.label_] = ent.text
        second_area = address.split()[1]
        print("second_area",second_area )
        df = load_data_into_dataframe(f"SELECT * FROM naver_complexes WHERE cortar_address LIKE '%{second_area}%' and real_estate_type_name not like '%분양권'")
        #TODO: 좌표로 찾는 방법을 실험해야함.

        #Same area Gu or Shi
        # display(df)
        src_values = entities.get('건물') or entities.get('단지')
        print("src_values", src_values) # apt name or building name
        if src_values:
            closest_df = find_closest_building(src_values, df)
            # If the dataframe is not empty, get the row with the highest similarity
            if not closest_df.empty:
                # Pass the complex_no, entities.get('건물동'), and entities.get('건물호') to the search_complex_in_naver_data
                df_price=search_complex_price(closest_df['complex_no'], entities.get('건물동'), entities.get('건물호'))
                return df_price, closest_df['complex_no'], entities.get('건물동'), entities.get('건물호')
            else:
                return src_values, entities.get('건물동'), entities.get('건물호')
    return None,None,None,None


def find_complex_price_by_cord(address):
    closest_df=None
    if address is not None:
    # Analyze the address with the NLP model
        doc = nlp(address.strip())

        # Create a dictionary to store the entities for this address
        entities = {'address': address.strip()}
        # Add the entities to the dictionary
        for ent in doc.ents:
            entities[ent.label_] = ent.text
        second_area = address.split()[1]




        print("second_area",second_area )
        df = load_data_into_dataframe(f"SELECT * FROM naver_complexes WHERE cortar_address LIKE '%{second_area}%' and real_estate_type_name not like '%분양권'")
        #TODO: 좌표로 찾는 방법을 실험해야함.

        #Same area Gu or Shi
        # display(df)
        src_values = entities.get('건물') or entities.get('단지')
        print("src_values", src_values) # apt name or building name
        if src_values:
            closest_df = find_closest_building(src_values, df)
            # If the dataframe is not empty, get the row with the highest similarity
            if not closest_df.empty:
                # Pass the complex_no, entities.get('건물동'), and entities.get('건물호') to the search_complex_in_naver_data
                df_price=search_complex_price(closest_df['complex_no'], entities.get('건물동'), entities.get('건물호'))
                return df_price, closest_df['complex_no'], entities.get('건물동'), entities.get('건물호')
            else:
                return src_values, entities.get('건물동'), entities.get('건물호')
    return None,None,None,None

def on_button_clicked(b):
    clear_output(wait=True)  # Clear the previous output
    address = text.value
    df_price, cn, bd,bh =find_complex_price(address)

    if not df_price.empty:
        display(text, button)
        display(address)
        display( cn, bd,bh)
        display(df_price)
    else :
        display(text, button)
        display(address)
        display( cn, bd,bh)
        display('Complex price Not found')


text = widgets.Text(
    value='',
    placeholder='Enter address line:',
    description='Address:',
    disabled=False,
    layout=widgets.Layout(width='500px')  # Adjust the width as needed

)

button = widgets.Button(
    description='찾기',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

display(text, button)
button.on_click(on_button_clicked)

/opt/conda/envs/py39/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Text(value='', description='Address:', layout=Layout(width='500px'), placeholder='Enter address line:')

Button(description='찾기', icon='check', style=ButtonStyle(), tooltip='Click me')

# Call price

In [40]:
import argparse
import numpy as np
import random
import requests
from qrytool import load_data_into_dataframe, insert_dataframe_into_table
import json
import time
import pandas as pd
from icecream import ic
import re
def camel_to_snake(name):
    name = re.sub("(.)([A-Z][a-z]+)", r"\1_\2", name)
    return re.sub("([a-z0-9])([A-Z])", r"\1_\2", name).lower()


def get_random_time():
    return random.uniform(0.1, 0.8)


def read_proxies_from_file(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()
    proxies = [
        {"ip": line.split(":")[0].strip(), "port": line.split(":")[1].strip()}
        for line in lines
    ]
    return proxies


def get_proxies():
    proxies = read_proxies_from_file("/workspace/proxies.txt")
    proxy_list = []
    for proxy in proxies:
        if proxy is not None:
            # print("Found proxy: %s" % proxy)
            proxy_list.append(proxy)
    return proxy_list


def get_random_proxy():
    global proxies_list
    proxy = random.choice(proxies_list)
    time.sleep(get_random_time())
    return {"http": f"http://{proxy['ip']}:{proxy['port']}"}


def fetch_data(url, complex_no):
    proxy = get_random_proxy()
    ic(proxy, url)
    try:
        response = requests.get(
            url,
            proxies=proxy,
            headers={
                "Accept-Encoding": "gzip",
                "Host": "new.land.naver.com",
                        "Referer": "https://new.land.naver.com/complexes/"+complex_no+"?ms=37.474847,127.10508,17&a=OPST:OBYG:PRE&e=RETAIL" ,
                        "Sec-Fetch-Dest": "empty",
                        "Sec-Fetch-Mode": "cors",
                        "Sec-Fetch-Site": "same-origin",
                        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36",
            },

        )
        return response
    except requests.exceptions.RequestException as e:
        print(f"Request failed with proxy {proxy}. Retrying... Error: {e}")
        return fetch_data(url)  # 재시도


def apt_call_price(complex_no):
    # pyeong_no = str(pyeong_no)
    complex_no = str(complex_no)
    # p_num = temp["complexPyeongDetailList"][index]["pyeongNo"]
    # p_num = str(index)
    # down_url = f"https://new.land.naver.com/api/articles/complex/{complex_no}?realEstateType=OPST%3AAPT%3APRE&tradeType=&tag=%3A%3A%3A%3A%3A%3A%3A%3A&rentPriceMin=0&rentPriceMax=900000000&priceMin=0&priceMax=900000000&areaMin=0&areaMax=900000000&oldBuildYears&recentlyBuildYears&minHouseHoldCount&maxHouseHoldCount&showArticle=false&sameAddressGroup=false&minMaintenanceCost&maxMaintenanceCost&priceType=RETAIL&directions=&page=1&complexNo={complex_no}&buildingNos=&areaNos=&type=list&order=rank"
    down_url = f"https://new.land.naver.com/api/articles/complex/{complex_no}?realEstateType=OPST%3AAPT%3APRE&tradeType=&oldBuildYears&recentlyBuildYears&minHouseHoldCount&maxHouseHoldCount&showArticle=false&sameAddressGroup=false&minMaintenanceCost&maxMaintenanceCost&priceType=RETAIL&directions=&page=1&complexNo={complex_no}&buildingNos=&areaNos=&type=list&order=rank"
            #    f"https://new.land.naver.com/api/articles/complex/{complex_no}?realEstateType=OPST%3AAPT%3APRE&tradeType=&tag=%3A%3A%3A%3A%3A%3A%3A%3A&rentPriceMin=0&rentPriceMax=900000000&priceMin=0&priceMax=900000000&areaMin=0&areaMax=900000000&oldBuildYears&recentlyBuildYears&minHouseHoldCount&maxHouseHoldCount&showArticle=false&sameAddressGroup=false&minMaintenanceCost&maxMaintenanceCost&priceType=RETAIL&directions=&page=1&complexNo=106710&buildingNos=&areaNos=&type=list&order=rank"
    r = fetch_data(down_url, complex_no)
    r.encoding = "utf-8-sig"
    print(r.text)  # Add this line
    if r.text is None or r.text.strip() == '':
        return None
    r.encoding = "utf-8-sig"
    temp_price = json.loads(r.text)
    return temp_price

proxies_list = get_proxies()
apt_call_price(106710)

ic| proxy: {'http': 'http://162.223.94.166:80'}
    url: 'https://new.land.naver.com/api/articles/complex/106710?realEstateType=OPST%3AAPT%3APRE&tradeType=&oldBuildYears&recentlyBuildYears&minHouseHoldCount&maxHouseHoldCount&showArticle=false&sameAddressGroup=false&minMaintenanceCost&maxMaintenanceCost&priceType=RETAIL&directions=&page=1&complexNo=106710&buildingNos=&areaNos=&type=list&order=rank'


{"isMoreData":true,"articleList":[{"articleNo":"2420025398","articleName":"강남힐스테이트에코","articleStatus":"R0","realEstateTypeCode":"OPST","realEstateTypeName":"오피스텔","articleRealEstateTypeCode":"A02","articleRealEstateTypeName":"오피스텔","tradeTypeCode":"B2","tradeTypeName":"월세","verificationTypeCode":"NDOC1","floorInfo":"6/10","rentPrc":"80","priceChangeState":"SAME","isPriceModification":false,"dealOrWarrantPrc":"1,000","areaName":"48A-1","area1":48,"area2":23,"direction":"남향","articleConfirmYmd":"20240424","siteImageCount":0,"articleFeatureDesc":"정류장 바로앞,쾌적한환경","tagList":["10년이내","소형평수","화장실한개","방한개"],"buildingName":"1동","sameAddrCnt":1,"sameAddrDirectCnt":0,"sameAddrMaxPrc":"1,000/80","sameAddrMinPrc":"1,000/80","cpid":"hkdotcom","cpName":"한경부동산","cpPcArticleUrl":"https://realestate.hankyung.com/naver-listing?UID=2420025398","cpPcArticleBridgeUrl":"","cpPcArticleLinkUseAtArticleTitleYn":false,"cpPcArticleLinkUseAtCpNameYn":true,"cpMobileArticleUrl":"","cpMobileArticleLinkUseAtArticleTitl

{'isMoreData': True,
 'articleList': [{'articleNo': '2420025398',
   'articleName': '강남힐스테이트에코',
   'articleStatus': 'R0',
   'realEstateTypeCode': 'OPST',
   'realEstateTypeName': '오피스텔',
   'articleRealEstateTypeCode': 'A02',
   'articleRealEstateTypeName': '오피스텔',
   'tradeTypeCode': 'B2',
   'tradeTypeName': '월세',
   'verificationTypeCode': 'NDOC1',
   'floorInfo': '6/10',
   'rentPrc': '80',
   'priceChangeState': 'SAME',
   'isPriceModification': False,
   'dealOrWarrantPrc': '1,000',
   'areaName': '48A-1',
   'area1': 48,
   'area2': 23,
   'direction': '남향',
   'articleConfirmYmd': '20240424',
   'siteImageCount': 0,
   'articleFeatureDesc': '정류장 바로앞,쾌적한환경',
   'tagList': ['10년이내', '소형평수', '화장실한개', '방한개'],
   'buildingName': '1동',
   'sameAddrCnt': 1,
   'sameAddrDirectCnt': 0,
   'sameAddrMaxPrc': '1,000/80',
   'sameAddrMinPrc': '1,000/80',
   'cpid': 'hkdotcom',
   'cpName': '한경부동산',
   'cpPcArticleUrl': 'https://realestate.hankyung.com/naver-listing?UID=2420025398',
   'cp